In [1]:
import math
import random
import time
import matplotlib.pyplot as plt 
import datetime
from multiprocessing import Pool
from itertools import combinations, product
from statistics import mean, stdev
from itertools import combinations
from numpy import log, append
from collections import deque # Cola FIFO para la lista tabú
from numpy import argsort # Ordenar lista por indices (GRASP)

verbose = False

# Funciones comunes

In [6]:
def leeFichero(nombreFichero ='tsp/a280.tsp'):
    rutaFichero = './tsp/'
    extension = '.tsp'
    fichero = open(rutaFichero + nombreFichero + extension, 'r')
    array_puntos = []

    # Saltamos las cabeceras del fichero #
    for i in range(6):
        fichero.readline()

    for line in fichero:
        try:
            array_temp = line.replace('   ', ' ').replace('  ', ' ').replace('\n', '').split(' ')
            array_temp = [float(item) for item in array_temp if item]

            if(len(array_temp) > 1):
                array_puntos.append(array_temp)
        except:
            pass

    fichero.close()

    return array_puntos

In [7]:
def getDistancia(p1, p2):
    xd = p1[1] - p2[1]
    yd = p1[2] - p2[2]
    
    dij = round(math.sqrt(xd*xd + yd*yd))
    
    return dij

In [8]:
def getMDistancias(nodos):
   
    distancias = [[0]*len(nodos) for i in range(len(nodos))]
    
    for i in range(len(nodos)):
        for j in range(len(nodos)):
            
            if(i==j):
                dij = 0 # Distancia a sí mismo infinita
            else:
                dij = getDistancia(nodos[i], nodos[j])
                
            distancias[i][j] = dij
            
    return distancias

In [9]:
def pintaCamino(camino, nodos, titulo):
    
    x = []
    y = []
    
    for i in range(0, len(camino)-1):
        
        x.append(nodos[camino[i]-1][1])
        y.append(nodos[camino[i]-1][2])
        
    plt.figure(figsize=(12,12))    
    plt.title(titulo)
    plt.scatter(x, y, color='green') 
    
    for i in range(len(x)):
        if(i < len(x)-1):
            plt.arrow(x[i], y[i], x[i+1] - x[i], y[i+1] - y[i], head_width=1, length_includes_head=True)
        else:
             plt.arrow(x[i], y[i], x[0] - x[i], y[0] - y[i], head_width=1, length_includes_head=True)
                
    # Punto rojo inicial
    plt.plot(x[0],y[0], marker='o',
     markerfacecolor='red', markersize=12)
                

In [10]:
def getCosteCamino(camino, distancias, verbose=False):
    coste = 0
    
    for i in range(0, len(camino)-1):
        dij = distancias[camino[i]][camino[i+1]]
        coste += dij # Sumamos la distancia de un nodo al siguiente
        if(verbose):
            print(f"Acumulando Distancia {camino[i]} -> {camino[i+1]} = {dij}")
        
        
    dij = distancias[camino[-1]][camino[0]] #Sumamos la distancia del ultimo al primero
    coste += dij
    if(verbose):
            print(f"Acumulando Distancia {camino[-1]} -> {camino[0]} = {dij}")
    
    return coste

In [131]:
def getCostePivotes(distancias, caminoAnt, costeAnt, piv1, piv2):
      
    caminoAnt1 = [caminoAnt[piv1-1], caminoAnt[piv1]]
    if(piv1 != len(distancias)-1):
        caminoAnt1.append(caminoAnt[piv1+1])
    else:
        # Pivote es el último elemento: último al primero
        caminoAnt1.append(caminoAnt[0])
        
    caminoAnt2 = [caminoAnt[piv2-1], caminoAnt[piv2]]
    if(piv2 != len(distancias)-1):
        caminoAnt2.append(caminoAnt[piv2+1])
    else:
        # Pivote es el último elemento: último al primero
        caminoAnt2.append(caminoAnt[0])
        
    caminoNuevo1 = caminoAnt1.copy()
    if caminoAnt[piv2] in caminoNuevo1:
        caminoNuevo1[caminoNuevo1.index(caminoAnt[piv2])] = caminoAnt[piv1]
    caminoNuevo1[1] = caminoAnt[piv2]
    
    
    caminoNuevo2 = caminoAnt2.copy()
    if caminoAnt[piv1] in caminoNuevo2:
        caminoNuevo2[caminoNuevo2.index(caminoAnt[piv1])] = caminoAnt[piv2]
    caminoNuevo2[1] = caminoAnt[piv1]
   
    #print("SubCaminos anteriores: ", caminoAnt1, caminoAnt2)
    #print("SubCaminos nuevos: ", caminoNuevo1, caminoNuevo2)
    
    return (costeAnt - distancias[caminoAnt1[0]][caminoAnt1[1]] - distancias[caminoAnt1[1]][caminoAnt1[2]] - distancias[caminoAnt2[0]][caminoAnt2[1]] - distancias[caminoAnt2[1]][caminoAnt2[2]] + distancias[caminoNuevo1[0]][caminoNuevo1[1]] +  distancias[caminoNuevo1[1]][caminoNuevo1[2]] + distancias[caminoNuevo2[0]][caminoNuevo2[1]] + distancias[caminoNuevo2[1]][caminoNuevo2[2]])
    

# Algoritmo Greedy

In [132]:
# Algoritmo Greedy
def algoritmoGreedy(distancias):
    
    caminoSolucion = [0] # Empezamos en el primer indice de nodo
    nodosPendientes = list(range(1,len(distancias[1]))) # Lista del segundo al ultimo indice de nodo
    
    while len(nodosPendientes) > 0: # Mientras queden nodos por visitar

        nodoActual = caminoSolucion[-1] # Partimos desde el útlimo nodo añadido
        minDistancia = distancias[nodoActual][nodosPendientes[0]] # Inicializamos la distancia del actual al primer pendiente
        nodoMinDistancia = nodosPendientes[0]
        
        for nodoVecino in nodosPendientes: # Recorremos todos los nodos pendientes y nos quedamos con el de menor distancia
            d = distancias[nodoActual][nodoVecino]
            
            if d < minDistancia:
                minDistancia = d
                nodoMinDistancia = nodoVecino
                
        caminoSolucion.append(nodoMinDistancia)
        nodosPendientes.remove(nodoMinDistancia)
        
    return caminoSolucion
    #return [indice+1 for indice in caminoSolucion] # + 1 a cada elemento (tiene que empezar por 1)

## Búsqueda Aleatoria

In [133]:
# Busqueda aleatoria: el mejor camino de entre 1600n soluciones
def busquedaAleatoria(distancias, semilla, nIteraciones):
    
    random.seed = semilla # Aplicamos la semilla pasada por parámetro
    mejorCamino = []
    mejorCoste = -1
    
    for i in range (nIteraciones):
        caminoSolucion = [random.randint(1,len(distancias))] # Empezamos en un nodo aleatorio
        nodosPendientes = [1+indice for indice in range(len(distancias))] # Lista del 1 al num. nodos
        nodosPendientes.remove(caminoSolucion[0]) # Eliminamos el primer nodo visitado

        while len(nodosPendientes) > 0: # Mientras queden nodos por visitar

            nodoActual = caminoSolucion[-1] # Partimos desde el útlimo nodo añadido
            indiceAleatorio = random.randint(0,len(nodosPendientes)-1)
            nodoSiguiente = nodosPendientes[indiceAleatorio]

            caminoSolucion.append(nodoSiguiente)
            nodosPendientes.pop(indiceAleatorio)
        
        # Calculamos el coste de esa solucion
        costeActual = getCosteCamino(caminoSolucion, distancias)
        
        # La primera vez o si se reduce el coste, nos quedamos con la solucion
        if(mejorCoste==-1 or costeActual < mejorCoste): 
            mejorCamino = caminoSolucion
            mejorCoste = costeActual
        
    return [mejorCamino, mejorCoste, semilla]

In [134]:
def busquedaAleatoria_v2(distancias, semilla, nIteraciones):
    random.seed = semilla # Aplicamos la semilla pasada por parámetro
    numNodos = len(distancias)
    
    # Solución inicial
    mejorCamino = random.sample(range(numNodos), numNodos) # Combinación aleatoria sin repeteción de nodos
    mejorCoste = getCosteCamino(mejorCamino, distancias)
    
    for i in range (nIteraciones):
        
        caminoCandidato = random.sample(range(numNodos), numNodos) # Combinación aleatoria sin repeteción de nodos
        # Calculamos el coste de esa solucion
        costeActual = getCosteCamino(caminoCandidato, distancias)
        
        # Si se reduce el coste, nos quedamos con la solucion
        if(costeActual < mejorCoste): 
            mejorCamino = caminoCandidato
            mejorCoste = costeActual
        
    return [mejorCamino, mejorCoste, semilla]

## Búsqueda Local del Mejor Vecino

In [135]:
# Obtiene una lista de todos los caminos vecinos que se obtienen al permutar dos elementos del camino dado
def solucionesVecinas(camino):
    numNodos = len(camino)
    indices = list(range(numNodos))
    vecinos = []


    # Todas las combinaciones de dos elementos (cambios posibles)
    comb = combinations(indices, 2)  

    # Aplicamos operador opt-2
    for cambio in comb:
        camino_new = camino.copy()

        # Permutamos los dos elementos
        camino_new[cambio[0]], camino_new[cambio[1]] = camino_new[cambio[1]], camino_new[cambio[0]]

        vecinos.append(camino_new)
    
    return vecinos

In [136]:
def busquedaLocalMejorVecino(distancias, semilla, nIteraciones, verbose=False):
     
    random.seed = semilla # Aplicamos la semilla pasada por parámetro
  
    numNodos = len(distancias)
    solucionActual = random.sample(range(numNodos), numNodos) # Combinación aleatoria sin repeteción de nodos
    costeActual = getCosteCamino(solucionActual, distancias) 
    
    i=0
    
    repetir = True
    while repetir:
        
        mejorVecino = solucionActual.copy()
        costeMejorVecino = costeActual
        
        for solucionVecina in solucionesVecinas(solucionActual):
            
            costeSolucionVecina = getCosteCamino(solucionVecina, distancias)
            i+=1
            
            if(costeSolucionVecina < costeMejorVecino):
                mejorVecino = solucionVecina
                costeMejorVecino = costeSolucionVecina
                    
                
        if(costeMejorVecino < costeActual):
            solucionActual = mejorVecino.copy()
            costeActual = costeMejorVecino
            
        # Condición de parada
        elif(costeMejorVecino >= costeActual):
           # print("No mejora la exploración de vecinos con i: %i !" % i)
            repetir = False
            
        
        if(i > nIteraciones):
            #print("Alcanzado limite de iteraciones: %i !" % i)
            repetir = False
        
            
    return [solucionActual, costeActual, i, semilla]

In [137]:
def busquedaLocalMejorVecino_v2(distancias, semilla, nIteraciones, solucionInicial=False):
    random.seed = semilla # Aplicamos la semilla pasada por parámetro
    
    numNodos = len(distancias)
    indices = list(range(numNodos)) # Conversión a list para poderlo iterar +1 vez
    
    # Todas las combinaciones de dos elementos (cambios posibles)
    comb = list(combinations(indices, 2))  
    
    if not solucionInicial:
        solucionActual = random.sample(range(numNodos), numNodos) # Combinación aleatoria sin repeteción de nodos
    else:
        solucionActual = solucionInicial
        
    costeActual = getCosteCamino(solucionActual, distancias) 

    i=0
    repetir = True
    while repetir:
        mejorVecino = solucionActual
        costeMejorVecino = costeActual
    
        # Por cada posible camino vecino
        for cambio in comb:
            
            costeSolucionVecina = getCostePivotes(distancias, solucionActual, costeActual, cambio[0], cambio[1])
            #print("cambio " + str(cambio[0]) + " ; " + str(cambio[1]))
            i+=1

            if(costeSolucionVecina < costeMejorVecino):
                #print("Mejora la solucion vecina %f < %f" % (costeSolucionVecina, costeMejorVecino) )
                
                # Aplicamos operador opt-2
                camino_new = solucionActual.copy()
                camino_new[cambio[0]], camino_new[cambio[1]] = camino_new[cambio[1]], camino_new[cambio[0]]
                
                mejorVecino = camino_new
                costeMejorVecino = costeSolucionVecina
                
                
        if(costeMejorVecino < costeActual):
            #print("Mejora la solucion actual %f < %f" % (costeMejorVecino, costeActual) )
            solucionActual = mejorVecino
            costeActual = costeMejorVecino
            
        # Condición de parada
        elif(costeMejorVecino >= costeActual):
            #print("No mejora coste con i: %i !" % i)
            repetir = False
            
        if(i > nIteraciones):
            #print("Alcanzado limite de iteraciones: %i !" % i)
            repetir = False   
    return [solucionActual, costeActual, i, semilla]

In [138]:
def busquedaLocalPrimerMejorVecino(distancias, semilla, nIteraciones, verbose=False):
     
    random.seed = semilla # Aplicamos la semilla pasada por parámetro
    
    numNodos = len(distancias)
    solucionActual = random.sample(range(numNodos), numNodos) # Combinación aleatoria sin repeteción de nodos
    costeActual = getCosteCamino(solucionActual, distancias) 
    i=0
    
    repetir = True
    while repetir:
        
        mejorVecino = solucionActual
        costeMejorVecino = costeActual
        
        solVecinas = solucionesVecinas(solucionActual) 

        
        for solucionVecina in solVecinas:
            
            #t = time.time()
            costeSolucionVecina = getCosteCamino(solucionVecina, distancias)  # hacer funcion coste reducida cambiando dos posiciones
            #elapsed = time.time() - t
            #print("Tiempo calcular coste solucion vecina: %f\n" % elapsed )
            i+=1
            
            if(costeSolucionVecina < costeMejorVecino):
                print("Mejora la solucion vecina %f < %f" % (costeSolucionVecina, costeMejorVecino) )
                mejorVecino = solucionVecina
                costeMejorVecino = costeSolucionVecina
                break # Paramos en el primer mejor vecino 
                
                
        if(costeMejorVecino < costeActual):
            print("Mejora la solucion actual %f < %f" % (costeMejorVecino, costeActual) )
            solucionActual = mejorVecino
            costeActual = costeMejorVecino
            
        # Condición de parada
        elif(costeMejorVecino >= costeActual):
           #print("No mejora coste con i: %i !" % i)
            repetir = False
            
        if(i > nIteraciones):
            #print("Alcanzado limite de iteraciones: %i !" % i)
            repetir = False   
        
    return [solucionActual, costeActual, i, semilla]

# Algoritmo Genético Básico

## Estructura de datos

In [1]:
class Cromosoma:
    
    # Métodos de clase
    def set_nodos_distancias(num_nodos, distancias):
        Cromosoma.num_nodos = num_nodos
        Cromosoma.distancias = distancias
    
    # Métodos de instancias
    
    # Operador minimo
    def __lt__(self, other):
        return self.coste < other.coste
    
    # Se puede instanciar de 3 maneras:
    # Cromosoma(): solucion aleatoria no elite
    # Cromosoma(camino, coste): solucion y coste definidos, no elite
    def __init__(self, *args):
        
        if not Cromosoma.num_nodos or not Cromosoma.distancias:
            raise TypeError("No se ha inicializado el num nodos o las distancias para la clase Cromosoma") 
        
        if len(args) == 0:
            # Combinación aleatoria sin repeteción de nodos
            self.camino = random.sample(range(Cromosoma.num_nodos), Cromosoma.num_nodos) # Combinación aleatoria sin repeteción de nodos
            self.coste = getCosteCamino(self.camino, Cromosoma.distancias) 
            
        elif len(args) == 2:
            self.camino = args[0]
            self.coste = args[1]        
        else:
            raise TypeError("Número de argumentos no válido, solo 0 ó 2") 
            
    def set_camino(self, camino):
        self.camino = camino
        
    def set_coste(self, coste):
        self.camino = coste
        
    def set_elite(self, elite):
        self.elite = elite
        
    def get_camino(self):
        return self.camino
    
    def get_coste(self):
        return self.coste
        
    

In [ ]:
class Poblacion:
    
    def __init__(self): 
        self.coste = 0
        self.media = 0
        self.individuos = []
    
    def add(self, cromosoma):
        self.individuos.append(cromosoma)
        self.coste += cromosoma.coste
        self.media = coste / len(individuos)
    
    def remove(self, cromosoma):
        self.individuos.remove(cromosoma)
        self.coste -= cromosoma.coste
        self.media = coste / len(individuos)
        del cromosoma
    

## Funciones auxiliares

In [ ]:
def seleccion_torneo(poblacion, k):
    
    # Seleccionar k individuos aleatorios
    individuos = random.sample(poblacion.individuos, k)
    
    # Devolver el mejor
    return min(individuos)

In [ ]:
# Operador de Cruce: basado en Orden OX
def cruce_OX(padre1, padre2):
    
    # Mantenemos una sublista del primer padre
    # De tamaño 50%
    tama = round(0.5 * len(padre1))
    
    pivote1 = random.randint()
    
    

## Algoritmo

In [ ]:
def genetico_basico(semilla, num_nodos, distancias, tam_poblacion, k_torneo):
    
    # Aplicamos la semilla
    random.seed = semilla
    
    # Inicializamos los atributos de la clase Cromosoma
    Cromosoma.num_nodos = num_nodos
    Cromosoma.distancias = distancias
    
    # Creamos la población inicial
    poblacion = Poblacion()
    for i in range(tam_poblacion-1):
        poblacion.add(Cromosoma())
    
    # Sembramos una solución greedy en la población
        camino_greedy = algoritmoGreedy(distancias)
        coste_greedy = getCosteCamino(camino_greedy, distancias)
        poblacion.add(Cromosoma(camino_greedy, coste_greedy))
        
        
    seguir = True
    while(seguir):
        
        # Parámetro k del torneo: 10% población
        k = 0.1 * len(poblacion.individuos)
        
        # Seleccionamos dos padres mediante torneo
        padres = []
        padre1 = seleccion_torneo(poblacion, k)
        padre2 = seleccion_torneo(poblacion, k)
        
        
        
        
        
    